# Performance Tests for our Modified Ray-Tracer
Here we present the tests used to generate the performance results quoted in our report.  Please see the report for more details and discussion.

#### Contents
1. Setup
1. Performance vs. Sample Fraction
1. Performance vs. Filter Parameters
1. Performance vs. Number of Pixels
1. Performance vs. Number of Objects
1. Performance vs. Number of Lights

## 1: Setup

In [ ]:
##Import necessary modules.
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm as logger
from mpl_toolkits.axes_grid1 import make_axes_locatable as axloc
import func_MRT as mrt
import time as timer
pi = np.pi
plt.close()

##Set booleans.
dosave = False #If True, will save rather than display all performance tests

## 2: Performance vs. Sample Fraction

In [ ]:
##Generate the worlds and scenes for this test.
#Generate lights
light1 = mrt.Light3D_Point(valdict={"coord":np.array([10.0,10.0,2.0]), "scaler":50.0,
                                "color_norm":1.0, "path":None, "iseverywhere":False})
light2 = mrt.Light3D_Point(valdict={"coord":np.array([10.0,1.0,0.0]), "scaler":75.0,
                                "color_norm":1.0, "path":None, "iseverywhere":False})
light3 = mrt.Light3D_Point(valdict={"coord":np.array([-4.0,3.0,0.0]), "scaler":25.0,
                                "color_norm":1.0, "path":None, "iseverywhere":False})
#Generate objects
eps = 1E-5
col1 = np.array([0.2, 1.0, 0.0])
sphere1 = mrt.Sphere3D(valdict={"coord":np.array([-0.5,0.0,0.0]), "rad":2.5, "color":col1, "eps":eps})
col2 = np.array([0.0, 1.0, 0.5])
sphere2 = mrt.Sphere3D(valdict={"coord":np.array([-10.0,0.0,0.0]), "rad":5.5, "color":col2, "eps":eps})
path1raw = np.array([0.0,1.0,0.0])
path1 = path1raw/np.linalg.norm(path1raw)
plane1 = mrt.Plane3D(valdict={"coord":np.array([0.0,-1.5,0.0]), "path":path1,
                          "color":np.array([0.7, 0.8, 0.2]), "eps":eps})
path2raw = np.array([0.0,-1.0,0.0])
path2 = path2raw/np.linalg.norm(path2raw)
plane2 = mrt.Plane3D(valdict={"coord":np.array([0.0,3.5,0.0]), "path":path2,
                          "color":np.array([1.0, 0.8, 0.2]), "eps":eps})
#Collect items into a scene
obj_list = [sphere1, sphere2, plane1, plane2]
light_list = [light1, light2, light3]

#Set scene parameters
pixsize = 0.1
hspan = 200
vspan = 200
czval = -25 #Camera's z-axis location (leave negative since window coordinates at origin)

#Generate a 'world'
world = mrt.World(obj_list=obj_list, light_list=light_list, backcolor=np.array([0,0,0]))
camera = mrt.Camera(coord=np.array([0,0,czval]), angles=np.array([0,0,0]))
window = mrt.Window(hspan=hspan, vspan=vspan, coord=np.array([0,0,0]), pixsize=pixsize)
scene = mrt.Scene(world=world, camera=camera, window=window)
imxy = np.array([window.coord[0]-(0.5*hspan), window.coord[0]+(0.5*hspan), window.coord[1]-(0.5*vspan),
                 window.coord[1]+(0.5*vspan)])*window.pixsize #For graphing later

In [ ]:
##Calculate performance for different sample fractions.
#Set variables for this test
saveroot = "test_frac"
frac_list = np.arange(0.025, 0.2505, 0.025) #Fraction of samples to use
numMRTs = len(frac_list)
sigma = 1
cutoff = 0.05

#Prepare lists to hold results
timetot_list = np.zeros(numMRTs)
res_list = [None]*numMRTs
score_list = np.zeros(numMRTs)
name_list = [None]*numMRTs
savename_list = [None]*numMRTs
sup_list = [None]*numMRTs

#Render the scene with the SRT
print("Running the SRT...")
timestart_st = timer.time() #Start the timer
renderer_st = mrt.Renderer_Standard(scene=scene, sampler=None, interper=None) #Generate the SRT
res_st = renderer_st.render_scene(form_shade="matte")["image"] #Run the SRT
timetot_st = timer.time() - timestart_st #Stop the timer
print("Finished running the SRT!")

#Render the scene with the MRT and different sample fractions
for ai in range(0, numMRTs):
    print("Running #{0} of 0-{1} MRTs...".format(ai, (numMRTs-1)))
    timestart_in = timer.time() #Start the timer
    #Generate sampler and interpolator
    sampler = mrt.Sampler_Fraction(valdict={"form":"grid", "fraction":frac_list[ai]}) #Fixed grid of samples
    interper = mrt.Interper_RBF(valdict={"verbose":True, "form":"linear", "max_inpoints":2000, "num_cappoints":10}) #Linear RBF
    #Generate and run the MRT
    renderer_in = mrt.Renderer_Interpolation(scene=scene, sampler=sampler, interper=interper,
                                                verbose=False, dofilter=True,
                                                sigma=sigma, cutoff=cutoff) #Generate the MRT
    res_list[ai] = renderer_in.render_scene(form_shade="matte")["image"] #Run the MRT
    timetot_list[ai] = timer.time() - timestart_in #Stop the timer
    print("Done running the {0}th MRT!".format(ai))
    #Store info strings for graphing later
    name_list[ai] = "MRT Results for {0}th Sample Fraction Test.".format(ai)
    savename_list[ai] = "{0}".format(ai)
    sup_list[ai] = ("{0}x{1} grid. Pix. size={2}. Sample Frac.={3:.3f}, Filter Sigma={4:.2f}, Filter Cut={5:.2f}."
                    .format(hspan, vspan, pixsize, frac_list[ai], sigma, cutoff))

In [ ]:
##Display performance results.
for ai in range(0, numMRTs):
    fig = plt.figure(figsize=(10,4))
    plt.axis("off")
    plt.suptitle("Results for Sample Fraction Test")
    plt.title(sup_list[ai])
    
    #Graph the SRT results
    print("Time for standard ray-tracer: {0}s.".format(timetot_st))
    ax0 = fig.add_subplot(1, 3, 1)
    pltmap = plt.imshow(res_st, extent=imxy, origin='lower')
    plt.title("SRT: {0:.2f}s.".format(timetot_st))
    ax0.xaxis.set_visible(False)
    ax0.yaxis.set_visible(False)
    
    #Graph the MRT results
    print("Time for {0}th modified ray-tracer: {1}s.".format(ai, timetot_list[ai]))
    ax0 = fig.add_subplot(1, 3, 2)
    pltmap = plt.imshow(res_list[ai], extent=imxy, origin='lower')
    plt.title("MRT: {0:.2f}s.".format(timetot_list[ai]))
    ax0.xaxis.set_visible(False)
    ax0.yaxis.set_visible(False)

    #Graph the error
    ax0 = fig.add_subplot(1, 3, 3)
    errgrid = np.sum(np.abs(res_list[ai] - res_st), axis=2)
    sumhere = errgrid.sum()
    score_list[ai] = sumhere
    pltmap = plt.imshow(errgrid/3.0, cmap=plt.cm.gnuplot2_r, origin='lower', norm=logger(vmin=1E-3, vmax=1))
    cbar = plt.colorbar(pltmap, cax=axloc(ax0).append_axes("right", size="5%", pad=0))
    cbar.set_label(r"$\sum_c$|Error|", rotation=270, labelpad=20)
    ax0.xaxis.set_visible(False)
    ax0.yaxis.set_visible(False)
    if dosave:
        plt.savefig("{0}_{1}.png".format(saveroot, savename_list[ai]))
        plt.close()
    else:
        plt.show()
    
#Graph the overall performance
#Plot the error
plt.title(r"Error/3N$_\mathrm{pix}$ vs. Sampling Fraction")
plt.plot(frac_list, (score_list/1.0/(3*hspan*vspan)), marker="o", color="black", linewidth=3, alpha=0.6)
plt.xlabel("Fraction used for Sampling")
plt.ylabel(r"$\sum_{x,y,c}$|Error|/3N$_\mathrm{pix}$")
plt.tight_layout()
if dosave:
    plt.savefig(saveroot+"_performance.png")
    plt.close()
else:
    plt.show()

#Plot the time
plt.title("Time vs. Sampling Fraction")
plt.semilogy(frac_list, (timetot_list/1.0/timetot_st), marker="o", color="blue", linewidth=3, alpha=0.6)
plt.ylim([0.01, 1])
plt.xlabel("Fraction used for Sampling")
plt.ylabel(r"(MRT Time / SRT Time)")
plt.tight_layout()
if dosave:
    plt.savefig(saveroot+"_time.png")
    plt.close()
else:
    plt.show()

## 3: Performance vs. Filter Parameters

In [ ]:
##Generate the worlds and scenes for this test.
#Generate lights
light1 = mrt.Light3D_Point(valdict={"coord":np.array([10.0,10.0,2.0]), "scaler":50.0,
                                "color_norm":1.0, "path":None, "iseverywhere":False})
light2 = mrt.Light3D_Point(valdict={"coord":np.array([10.0,1.0,0.0]), "scaler":75.0,
                                "color_norm":1.0, "path":None, "iseverywhere":False})
light3 = mrt.Light3D_Point(valdict={"coord":np.array([-4.0,3.0,0.0]), "scaler":25.0,
                                "color_norm":1.0, "path":None, "iseverywhere":False})
#Generate objects
eps = 1E-5
col1 = np.array([0.2, 1.0, 0.0])
sphere1 = mrt.Sphere3D(valdict={"coord":np.array([-0.5,0.0,0.0]), "rad":2.5, "color":col1, "eps":eps})
col2 = np.array([0.0, 1.0, 0.5])
sphere2 = mrt.Sphere3D(valdict={"coord":np.array([-10.0,0.0,0.0]), "rad":5.5, "color":col2, "eps":eps})
path1raw = np.array([0.0,1.0,0.0])
path1 = path1raw/np.linalg.norm(path1raw)
plane1 = mrt.Plane3D(valdict={"coord":np.array([0.0,-1.5,0.0]), "path":path1,
                          "color":np.array([0.7, 0.8, 0.2]), "eps":eps})
path2raw = np.array([0.0,-1.0,0.0])
path2 = path2raw/np.linalg.norm(path2raw)
plane2 = mrt.Plane3D(valdict={"coord":np.array([0.0,3.5,0.0]), "path":path2,
                          "color":np.array([1.0, 0.8, 0.2]), "eps":eps})
#Collect items into a scene
obj_list = [sphere1, sphere2, plane1, plane2]
light_list = [light1, light2, light3]

#Set scene parameters
pixsize = 0.1
hspan = 200
vspan = 200
czval = -25 #Camera's z-axis location (leave negative since window coordinates at origin)

#Generate a 'world'
world = mrt.World(obj_list=obj_list, light_list=light_list, backcolor=np.array([0,0,0]))
camera = mrt.Camera(coord=np.array([0,0,czval]), angles=np.array([0,0,0]))
window = mrt.Window(hspan=hspan, vspan=vspan, coord=np.array([0,0,0]), pixsize=pixsize)
scene = mrt.Scene(world=world, camera=camera, window=window)
imxy = np.array([window.coord[0]-(0.5*hspan), window.coord[0]+(0.5*hspan), window.coord[1]-(0.5*vspan),
                 window.coord[1]+(0.5*vspan)])*window.pixsize #For graphing later

In [ ]:
##Calculate performance for different filter parameters.
#Set variables for this test
saveroot = "test_filter"
frac = 0.1 #Fraction of samples to use
sigmaset = [1, 2, 4, 8]
cutoffset = [0.05, 0.1, 0.15, 0.2, 0.25]
sigma_list = np.repeat(sigmaset, len(cutoffset))
cutoff_list = np.tile(cutoffset, len(sigmaset))
numMRTs = len(sigma_list)

#Prepare lists to hold results
timetot_list = np.zeros(numMRTs)
res_list = [None]*numMRTs
score_list = np.zeros(numMRTs)
name_list = [None]*numMRTs
savename_list = [None]*numMRTs
sup_list = [None]*numMRTs

#Render the scene with the SRT
print("Running the SRT...")
timestart_st = timer.time() #Start the timer
renderer_st = mrt.Renderer_Standard(scene=scene, sampler=None, interper=None) #Generate the SRT
res_st = renderer_st.render_scene(form_shade="matte")["image"] #Run the SRT
timetot_st = timer.time() - timestart_st #Stop the timer
print("Finished running the SRT!")

#Render the scene with the MRT and different filter parameters
for ai in range(0, numMRTs):
    print("Running #{0} of 0-{1} MRTs...".format(ai, (numMRTs-1)))
    timestart_in = timer.time() #Start the timer
    #Generate sampler and interpolator
    sampler = mrt.Sampler_Fraction(valdict={"form":"grid", "fraction":frac}) #Fixed grid of samples
    interper = mrt.Interper_RBF(valdict={"verbose":True, "form":"linear", "max_inpoints":2000, "num_cappoints":10}) #Linear RBF
    #Generate and run the MRT
    renderer_in = mrt.Renderer_Interpolation(scene=scene, sampler=sampler, interper=interper,
                                                verbose=False, dofilter=True,
                                                sigma=sigma_list[ai], cutoff=cutoff_list[ai]) #Generate the MRT
    res_list[ai] = renderer_in.render_scene(form_shade="matte")["image"] #Run the MRT
    timetot_list[ai] = timer.time() - timestart_in #Stop the timer
    print("Done running the {0}th MRT!".format(ai))
    #Store info strings for graphing later
    name_list[ai] = "MRT Results for {0}th Sample Fraction Test.".format(ai)
    savename_list[ai] = "{0}".format(ai)
    sup_list[ai] = ("{0}x{1} grid. Pix. size={2}. Sample Frac.={3:.3f}, Filter Sigma={4:.2f}, Filter Cut={5:.2f}."
                    .format(hspan, vspan, pixsize, frac, sigma_list[ai], cutoff_list[ai]))

In [ ]:
##Display performance results.
for ai in range(0, numMRTs):
    fig = plt.figure(figsize=(10,4))
    plt.axis("off")
    plt.suptitle("Results for Filter Parameters Test")
    plt.title(sup_list[ai])
    
    #Graph the SRT results
    print("Time for standard ray-tracer: {0}s.".format(timetot_st))
    ax0 = fig.add_subplot(1, 3, 1)
    pltmap = plt.imshow(res_st, extent=imxy, origin='lower')
    plt.title("SRT: {0:.2f}s.".format(timetot_st))
    ax0.xaxis.set_visible(False)
    ax0.yaxis.set_visible(False)
    
    #Graph the MRT results
    print("Time for {0}th modified ray-tracer: {1}s.".format(ai, timetot_list[ai]))
    ax0 = fig.add_subplot(1, 3, 2)
    pltmap = plt.imshow(res_list[ai], extent=imxy, origin='lower')
    plt.title("MRT: {0:.2f}s.".format(timetot_list[ai]))
    ax0.xaxis.set_visible(False)
    ax0.yaxis.set_visible(False)
    
    #Graph the error
    ax0 = fig.add_subplot(1, 3, 3)
    errgrid = np.sum(np.abs(res_list[ai] - res_st), axis=2)
    sumhere = errgrid.sum()
    score_list[ai] = sumhere
    pltmap = plt.imshow(errgrid/3.0, cmap=plt.cm.gnuplot2_r, origin='lower', norm=logger(vmin=1E-3, vmax=1))
    cbar = plt.colorbar(pltmap, cax=axloc(ax0).append_axes("right", size="5%", pad=0))
    cbar.set_label(r"$\sum_c$|Error|", rotation=270, labelpad=20)
    ax0.xaxis.set_visible(False)
    ax0.yaxis.set_visible(False)
    if dosave:
        plt.savefig("{0}_{1}.png".format(saveroot, savename_list[ai]))
        plt.close()
    else:
        plt.show()
    
#Graph the overall performance
uniqsigma = np.unique(sigma_list)
markbank = ["o", "v", "s", "*", "D", "X", "^", "P", ">", "<"]
colorbank = ["black", "purple", "blue", "dodgerblue", "red", "orange", "darkgoldenrod", "green"]
markdict = {}
colordict = {}
for ai in range(0, len(uniqsigma)):
    markdict[uniqsigma[ai]] = markbank[ai]
    colordict[uniqsigma[ai]] = colorbank[ai]

#Plot the error
plt.title(r"Error/3N$_\mathrm{pix}$ vs. Filter Parameters")
for ai in range(0, numMRTs):
    plt.scatter(cutoff_list[ai], (score_list[ai]/(3.0*hspan*vspan)), marker=markdict[sigma_list[ai]],
                color=colordict[sigma_list[ai]], s=30, alpha=0.6)
for ai in range(0, len(uniqsigma)): #For legend
    plt.scatter([], [], marker=markdict[uniqsigma[ai]], color=colordict[uniqsigma[ai]],
                s=30, alpha=0.6, label=str(uniqsigma[ai]))
plt.xlabel("Filtering Cutoff")
plt.ylabel(r"$\sum_{x,y,c}$|Error|/3N$_\mathrm{pix}$")
plt.ylim([0.0025, 0.0100])
plt.legend(loc="best", title="Sigma")
plt.tight_layout()
if dosave:
    plt.savefig(saveroot+"_performance.png")
    plt.close()
else:
    plt.show()

#Plot the time
plt.title("Time vs. Filter Parameters")
for ai in range(0, numMRTs):
    plt.scatter(cutoff_list[ai], (timetot_list[ai]/1.0/timetot_st), marker=markdict[sigma_list[ai]],
                color=colordict[sigma_list[ai]], s=30, alpha=0.6)
for ai in range(0, len(uniqsigma)): #For legend
    plt.scatter([], [], marker=markdict[uniqsigma[ai]],
                color=colordict[uniqsigma[ai]], s=30, alpha=0.6, label=str(uniqsigma[ai]))
plt.xlabel("Filtering Cutoff")
plt.ylabel(r"(MRT Time / SRT Time)")
plt.yscale("log")
plt.ylim([0.01, 1])
plt.legend(loc="best", title="Sigma")
plt.tight_layout()
if dosave:
    plt.savefig(saveroot+"_time.png")
    plt.close()
else:
    plt.show()

## 4: Performance vs. Number of Pixels

In [ ]:
##Generate the parameters and objects for scenes in this test.
#Set parameters for scene generation
scale_list = np.arange(0.5, 3.05, 0.25) #Maintain zoom of scene.
numMRTs = len(scale_list)
eps = 1E-5

#Generate lights
light1 = mrt.Light3D_Point(valdict={"coord":np.array([10.0, 10.0, 10.0]), "scaler":80.0,
                                    "color_norm":1.0, "path":None, "iseverywhere":False})
light2 = mrt.Light3D_Point(valdict={"coord":np.array([0.0, 10.0, 2.0]), "scaler":70.0,
                                "color_norm":1.0, "path":None, "iseverywhere":False})
light3 = mrt.Light3D_Point(valdict={"coord":np.array([-8.0, -8.0, 5.0]), "scaler":65.0,
                                "color_norm":1.0, "path":None, "iseverywhere":False})
light4 = mrt.Light3D_Point(valdict={"coord":np.array([10.0, 0.0, -2.0]), "scaler":65.0,
                                "color_norm":1.0, "path":None, "iseverywhere":False})
#Generate spheres
sphere1 = mrt.Sphere3D(valdict={"coord":np.array([-7.0, -5.0, -5.0]), "rad":5.5,
                                "color":np.array([0.16, 0.62, 1.0]), "eps":eps})
sphere2 = mrt.Sphere3D(valdict={"coord":np.array([3.4, -6.0, 3.0]), "rad":2.0,
                                "color":np.array([0.38, 0.02, 1.0]), "eps":eps})
sphere3 = mrt.Sphere3D(valdict={"coord":np.array([-5.0, 6.0, 6.9]), "rad":2.5,
                                "color":np.array([1.0, 0.72, 0.26]), "eps":eps})
sphere4 = mrt.Sphere3D(valdict={"coord":np.array([0.0, 0.0, 0.0]), "rad":2.5,
                                "color":np.array([0.8, 0.7, 0.8]), "eps":eps})
sphere5 = mrt.Sphere3D(valdict={"coord":np.array([-10.0, 0.0, 0.0]), "rad":5.5,
                                "color":np.array([1.0, 0.77, 0.68]), "eps":eps})
#Generate planes
path1raw = np.array([0.2,0.5,1.0]) #Unnormalized "path" (i.e., geometric normal) of the plane
path1 = path1raw/np.linalg.norm(path1raw) #Normalized "path" of the plane
plane1 = mrt.Plane3D(valdict={"coord":np.array([0.0, 0.0, -10.0]), "path":path1,
                              "color":np.array([0.7, 0.5, 1.0]), "eps":eps})
path2raw = np.array([1.0,0.0,1.0])
path2 = path2raw/np.linalg.norm(path2raw)
plane2 = mrt.Plane3D(valdict={"coord":np.array([10.0, 0.0, -20.0]), "path":path2,
                              "color":np.array([0.5, 0.5, 0.5]), "eps":eps})
#Collect items into lists
obj_list = [sphere1, sphere2, sphere3, sphere4, sphere5, plane1, plane2]
light_list = [light1, light2, light3, light4]

##Generate each scene.
czval = -25 #Camera's z-axis location (leave negative since window coordinates at origin)
scene_list = [None]*numMRTs
imxy_list = [None]*numMRTs
numpix_list = [None]*numMRTs
hspan_list = [None]*numMRTs
vspan_list = [None]*numMRTs
pixsize_list = [None]*numMRTs
for ai in range(0, numMRTs):
    #Set current scene parameters
    pixsize_list[ai] = 0.05/(scale_list[ai])
    hspan_list[ai] = int(400*scale_list[ai])
    vspan_list[ai] = int(400*scale_list[ai])
    camera = mrt.Camera(coord=np.array([0,0,czval]), angles=np.array([0,0,0]))
    window = mrt.Window(hspan=hspan_list[ai], vspan=vspan_list[ai], coord=np.array([0,0,0]), pixsize=pixsize_list[ai])
    imxy_list[ai] = np.array([window.coord[0]-(0.5*hspan_list[ai]), window.coord[0]+(0.5*hspan_list[ai]),
                     window.coord[1]-(0.5*vspan_list[ai]),
                     window.coord[1]+(0.5*vspan_list[ai])])*window.pixsize #For graphing later
    numpix_list[ai] = hspan_list[ai]*vspan_list[ai]
    
    #Generate this world
    world = mrt.World(obj_list=obj_list, light_list=light_list, backcolor=np.array([0,0,0]))
    scene_list[ai] = mrt.Scene(world=world, camera=camera, window=window)

In [ ]:
##Calculate performance for different numbers of pixels.
#Set variables for this test
saveroot = "test_numpix"
frac = 0.1 #Fraction of samples to use
sigma = 1 #Sigma for id filter
cutoff = 0.05 #Cutoff for id filter

#Prepare lists to hold results
timetotin_list = np.zeros(numMRTs)
timetotst_list = np.zeros(numMRTs)
resin_list = [None]*numMRTs
resst_list = [None]*numMRTs
score_list = np.zeros(numMRTs)
name_list = [None]*numMRTs
savename_list = [None]*numMRTs
sup_list = [None]*numMRTs

#Render the different scenes with both SRT and MRT
for ai in range(0, numMRTs):
    #Render the scene with the SRT
    print("Running the SRT...")
    timestart_st = timer.time() #Start the timer
    renderer_st = mrt.Renderer_Standard(scene=scene_list[ai], sampler=None, interper=None) #Generate the SRT
    resst_list[ai] = renderer_st.render_scene(form_shade="matte")["image"] #Run the SRT
    timetotst_list[ai] = timer.time() - timestart_st #Stop the timer
    print("Finished running the SRT!")

    #Render the scene with the MRT
    print("Running #{0} of 0-{1} MRTs...".format(ai, (numMRTs-1)))
    timestart_in = timer.time() #Start the timer
    #Generate sampler and interpolator
    sampler = mrt.Sampler_Fraction(valdict={"form":"grid", "fraction":frac}) #Fixed grid of samples
    interper = mrt.Interper_RBF(valdict={"verbose":False, "form":"linear", "max_inpoints":2000, "num_cappoints":10}) #Linear RBF
    #Generate and run the MRT
    renderer_in = mrt.Renderer_Interpolation(scene=scene_list[ai], sampler=sampler, interper=interper,
                                                verbose=False, dofilter=True,
                                                sigma=sigma, cutoff=cutoff) #Generate the MRT
    resin_list[ai] = renderer_in.render_scene(form_shade="matte")["image"] #Run the MRT
    timetotin_list[ai] = timer.time() - timestart_in #Stop the timer
    print("Done running the {0}th MRT!".format(ai))
    
    #Store info strings for graphing later
    name_list[ai] = "Results for {0}th #Pixels Test.".format(ai)
    savename_list[ai] = "{0}".format(ai)
    sup_list[ai] = ("{0}x{1} grid. Pix. size={2:.4f}. Sample Frac.={3:.3f}, Filter Sigma={4:.2f}, Filter Cut={5:.2f}."
                    .format(hspan_list[ai], vspan_list[ai], pixsize_list[ai], frac, sigma, cutoff))

In [ ]:
##Display performance results.
for ai in range(0, numMRTs):
    fig = plt.figure(figsize=(10,4))
    plt.axis("off")
    plt.suptitle("Results for #Pixels Test ({0} Pixels)".format(numpix_list[ai]))
    plt.title(sup_list[ai])
    
    #Graph the SRT results
    print("Time for standard ray-tracer: {0}s.".format(timetotst_list[ai]))
    ax0 = fig.add_subplot(1, 3, 1)
    pltmap = plt.imshow(resst_list[ai], extent=imxy_list[ai], origin='lower')
    plt.title("SRT: {0:.2f}s.".format(timetotst_list[ai]))
    ax0.xaxis.set_visible(False)
    ax0.yaxis.set_visible(False)
    
    #Graph the MRT results
    print("Time for {0}th modified ray-tracer: {1}s.".format(ai, timetotin_list[ai]))
    #Graph the final image
    ax0 = fig.add_subplot(1, 3, 2)
    pltmap = plt.imshow(resin_list[ai], extent=imxy_list[ai], origin='lower')
    plt.title("MRT: {0:.2f}s.".format(timetotin_list[ai]))
    ax0.xaxis.set_visible(False)
    ax0.yaxis.set_visible(False)
    
    #Graph the error
    ax0 = fig.add_subplot(1, 3, 3)
    errgrid = np.sum(np.abs(resin_list[ai] - resst_list[ai]), axis=2)
    sumhere = errgrid.sum()
    score_list[ai] = sumhere
    pltmap = plt.imshow(errgrid/3.0, cmap=plt.cm.gnuplot2_r, origin='lower', norm=logger(vmin=1E-3, vmax=1))
    cbar = plt.colorbar(pltmap, cax=axloc(ax0).append_axes("right", size="5%", pad=0))
    cbar.set_label(r"$\sum_c$|Error|", rotation=270, labelpad=20)
    ax0.xaxis.set_visible(False)
    ax0.yaxis.set_visible(False)
    if dosave:
        plt.savefig("{0}_{1}.png".format(saveroot, savename_list[ai]))
        plt.close()
    else:
        plt.show()
    
#Graph the overall performance
#Plot the error
plt.title(r"Error/3N$_\mathrm{pix}$ vs. Number of Pixels")
plt.plot(numpix_list, score_list/1.0/(3*np.asarray(numpix_list)), marker="o", color="black", linewidth=3, alpha=0.6)
plt.xlabel("Number of Pixels")
plt.ylabel(r"$\sum_{x,y,c}$|Error|/3N$_\mathrm{pix}$")
plt.tight_layout()
if dosave:
    plt.savefig(saveroot+"_performance.png")
    plt.close()
else:
    plt.show()
#Plot the time
plt.title("Time vs. Number of Pixels")
plt.semilogy(numpix_list, (timetotin_list/1.0/timetotst_list), marker="o", color="blue", linewidth=3, alpha=0.6)
plt.ylim([0.01, 1])
plt.xlabel("Number of Pixels")
plt.ylabel(r"(MRT Time / SRT Time)")
plt.tight_layout()
if dosave:
    plt.savefig(saveroot+"_time.png")
    plt.close()
else:
    plt.show()

## Performance vs. Number of Objects

In [ ]:
##Generate the parameters and objects for scenes in this test.
#Set parameters for scene generation
finnum = 10 #Maximum number of spheres
count_list = np.arange(0, finnum+1, 1) #Count of spheres
numMRTs = len(count_list)

#Generate scene and world characteristics ahead of time
pixsize = 0.1
hspan = 200
vspan = 200
czval = 25 #Camera's z-axis location
camera = mrt.Camera(coord=np.array([0,0,czval]), angles=np.array([0,0,0]))
window = mrt.Window(hspan=hspan, vspan=vspan, coord=np.array([0,0,0]), pixsize=pixsize)
imxy = np.array([window.coord[0]-(0.5*hspan), window.coord[0]+(0.5*hspan),
                 window.coord[1]-(0.5*vspan), window.coord[1]+(0.5*vspan)])*window.pixsize #For graphing later

#Generate lights
light1 = mrt.Light3D_Point(valdict={"coord":np.array([0.0,0.0,-5.0]), "scaler":70.0,
                                "color_norm":1.0, "path":None, "iseverywhere":False})
light_list = [light1]

#Generate planes
eps = 1E-5
#For planes
path1raw = np.array([0.0,0.0,-1.0])
path1 = path1raw/np.linalg.norm(path1raw)
plane1 = mrt.Plane3D(valdict={"coord":np.array([0.0,0.0,3.0]), "path":path1,
                          "color":np.array([1.0, 1.0, 0.5]), "eps":eps})

##Generate each scene with different number of spheres
scene_list = [None]*numMRTs
#For ring of spheres
col = np.array([0.5, 0.3, 1.0])
rad = 2.0
for ai in range(0, numMRTs):
    numspheres = count_list[ai]
    obj_list = [plane1]
    #Add in new sphere(s) to the list of objects
    ang_list = np.linspace(0, 2*pi, numspheres+1)
    for bi in range(0, numspheres):
        coord = np.array([8*np.cos(ang_list[bi]), 8*np.sin(ang_list[bi]), 1.0])
        sphere = mrt.Sphere3D(valdict={"coord":coord, "rad":rad, "color":col, "eps":eps})
        obj_list.append(sphere)
    
    #Generate this world and scene
    world = mrt.World(obj_list=obj_list, light_list=light_list, backcolor=np.array([0,0,0]))
    scene_list[ai] = mrt.Scene(world=world, camera=camera, window=window)

In [ ]:
##Calculate performance for different numbers of objects.
#Set variables for this test
saveroot = "test_numobj"
frac = 0.1 #Fraction of samples to use
sigma = 1 #Sigma for id filter
cutoff = 0.05 #Cutoff for id filter

#Prepare lists to hold results
timetotin_list = np.zeros(numMRTs)
timetotst_list = np.zeros(numMRTs)
resin_list = [None]*numMRTs
resst_list = [None]*numMRTs
score_list = np.zeros(numMRTs)
name_list = [None]*numMRTs
savename_list = [None]*numMRTs
sup_list = [None]*numMRTs

#Render the different scenes with both SRT and MRT
for ai in range(0, numMRTs):
    #Render the scene with the SRT
    print("Running the SRT...")
    timestart_st = timer.time() #Start the timer
    renderer_st = mrt.Renderer_Standard(scene=scene_list[ai], sampler=None, interper=None) #Generate the SRT
    resst_list[ai] = renderer_st.render_scene(form_shade="matte")["image"] #Run the SRT
    timetotst_list[ai] = timer.time() - timestart_st #Stop the timer
    print("Finished running the SRT!")

    #Render the scene with the MRT
    print("Running #{0} of 0-{1} MRTs...".format(ai, (numMRTs-1)))
    timestart_in = timer.time() #Start the timer
    #Generate sampler and interpolator
    sampler = mrt.Sampler_Fraction(valdict={"form":"grid", "fraction":frac}) #Fixed grid of samples
    interper = mrt.Interper_RBF(valdict={"verbose":True, "form":"linear", "max_inpoints":2000, "num_cappoints":10}) #Linear RBF
    #Generate and run the MRT
    renderer_in = mrt.Renderer_Interpolation(scene=scene_list[ai], sampler=sampler, interper=interper,
                                                verbose=False, dofilter=True,
                                                sigma=sigma, cutoff=cutoff) #Generate the MRT
    resin_list[ai] = renderer_in.render_scene(form_shade="matte")["image"] #Run the MRT
    timetotin_list[ai] = timer.time() - timestart_in #Stop the timer
    print("Done running the {0}th MRT!".format(ai))
    
    #Store info strings for graphing later
    name_list[ai] = "Results for {0}th #Objects Test ({1} Spheres).".format(ai, count_list[ai])
    savename_list[ai] = "{0}".format(ai)
    sup_list[ai] = ("{0}x{1} grid. Pix. size={2}. Sample Frac.={3:.3f}, Filter Sigma={4:.2f}, Filter Cut={5:.2f}."
                    .format(hspan, vspan, pixsize, frac, sigma, cutoff))

In [ ]:
##Display performance results.
for ai in range(0, numMRTs):
    fig = plt.figure(figsize=(10,4))
    plt.axis("off")
    plt.suptitle("Results for #Objects Test ({0} Spheres)".format(count_list[ai]))
    plt.title(sup_list[ai])
    
    #Graph the SRT results
    print("Time for standard ray-tracer: {0}s.".format(timetotst_list[ai]))
    ax0 = fig.add_subplot(1, 3, 1)
    pltmap = plt.imshow(resst_list[ai], extent=imxy, origin='lower')
    plt.title("SRT: {0:.2f}s.".format(timetotst_list[ai]))
    ax0.xaxis.set_visible(False)
    ax0.yaxis.set_visible(False)
    
    #Graph the MRT results
    print("Time for {0}th modified ray-tracer: {1}s.".format(ai, timetotin_list[ai]))
    #Graph the final image
    ax0 = fig.add_subplot(1, 3, 2)
    pltmap = plt.imshow(resin_list[ai], extent=imxy, origin='lower')
    plt.title("MRT: {0:.2f}s.".format(timetotin_list[ai]))
    ax0.xaxis.set_visible(False)
    ax0.yaxis.set_visible(False)
    
    #Graph the error
    ax0 = fig.add_subplot(1, 3, 3)
    errgrid = np.sum(np.abs(resin_list[ai] - resst_list[ai]), axis=2)
    sumhere = errgrid.sum()
    score_list[ai] = sumhere
    pltmap = plt.imshow(errgrid/3.0, cmap=plt.cm.gnuplot2_r, origin='lower', norm=logger(vmin=1E-3, vmax=1))
    cbar = plt.colorbar(pltmap, cax=axloc(ax0).append_axes("right", size="5%", pad=0))
    cbar.set_label(r"$\sum_c$|Error|", rotation=270, labelpad=20)
    ax0.xaxis.set_visible(False)
    ax0.yaxis.set_visible(False)
    if dosave:
        plt.savefig("{0}_{1}.png".format(saveroot, savename_list[ai]))
        plt.close()
    else:
        plt.show()
    
#Graph the overall performance
#Plot the error
plt.title(r"Error/3N$_\mathrm{pix}$ vs. Number of Objects")
plt.plot(count_list, (score_list/1.0/(3*hspan*vspan)), marker="o", color="black", linewidth=3, alpha=0.6)
plt.xlabel("Number of Objects")
plt.ylabel(r"$\sum_{x,y,c}$|Error|/3N$_\mathrm{pix}$")
plt.tight_layout()
if dosave:
    plt.savefig(saveroot+"_performance.png")
    plt.close()
else:
    plt.show()
#Plot the time
plt.title("Time vs. Number of Objects")
plt.semilogy(count_list, (timetotin_list/1.0/timetotst_list), marker="o", color="blue", linewidth=3, alpha=0.6)
plt.ylim([0.01, 1])
plt.xlabel("Number of Objects")
plt.ylabel(r"(MRT Time / SRT Time)")
plt.tight_layout()
if dosave:
    plt.savefig(saveroot+"_time.png")
    plt.close()
else:
    plt.show()

## Performance vs. Number of Lights

In [ ]:
##Generate the parameters and objects for scenes in this test.
#Set parameters for scene generation
finnum = 20 #Maximum number of spheres
count_list = np.arange(1, finnum+1, 2) #Count of spheres
numMRTs = len(count_list)

#Generate scene and world characteristics ahead of time
pixsize = 0.1
hspan = 200
vspan = 200
czval = 25 #Camera's z-axis location
camera = mrt.Camera(coord=np.array([0,0,czval]), angles=np.array([0,0,0]))
window = mrt.Window(hspan=hspan, vspan=vspan, coord=np.array([0,0,0]), pixsize=pixsize)
imxy = np.array([window.coord[0]-(0.5*hspan), window.coord[0]+(0.5*hspan),
                 window.coord[1]-(0.5*vspan), window.coord[1]+(0.5*vspan)])*window.pixsize #For graphing later

#Generate objects
eps = 1E-5
#For planes
path1raw = np.array([0.0,0.0,-1.0])
path1 = path1raw/np.linalg.norm(path1raw)
plane1 = mrt.Plane3D(valdict={"coord":np.array([0.0,0.0,3.0]), "path":path1,
                          "color":np.array([1.0, 1.0, 0.5]), "eps":eps})
#For spheres
col = np.array([0.5, 0.3, 1.0])
rad = 2.0
sphere1 = mrt.Sphere3D(valdict={"coord":np.array([0.0,1.5,0.0]), "rad":rad, "color":col, "eps":eps})
sphere2 = mrt.Sphere3D(valdict={"coord":np.array([1.5,-1.25,0.0]), "rad":rad, "color":col, "eps":eps})
sphere3 = mrt.Sphere3D(valdict={"coord":np.array([-1.5,-1.25,0.0]), "rad":rad, "color":col, "eps":eps})
obj_list = [plane1, sphere1, sphere2, sphere3]

##Generate each scene with different number of lights
scene_list = [None]*numMRTs
#For ring of lights
for ai in range(0, len(count_list)):
    light_list = []
    numlights = count_list[ai]
    #Add in new sphere(s) to the list of objects
    ang_list = np.linspace(0, 2*pi, numlights+1)
    for bi in range(0, numlights):
        coord = np.array([8*np.cos(ang_list[bi]), 8*np.sin(ang_list[bi]), -4.0])
        light = mrt.Light3D_Point(valdict={"coord":coord, "scaler":20.0,
                                "color_norm":1.0, "path":None, "iseverywhere":False})
        light_list.append(light)
    
    #Generate this world and scene
    world = mrt.World(obj_list=obj_list, light_list=light_list, backcolor=np.array([0,0,0]))
    scene_list[ai] = mrt.Scene(world=world, camera=camera, window=window)

In [ ]:
##Calculate performance for different numbers of objects.
#Set variables for this test
saveroot = "test_numlit"
frac = 0.1 #Fraction of samples to use
sigma = 1 #Sigma for id filter
cutoff = 0.05 #Cutoff for id filter

#Prepare lists to hold results
timetotin_list = np.zeros(numMRTs)
timetotst_list = np.zeros(numMRTs)
resin_list = [None]*numMRTs
resst_list = [None]*numMRTs
score_list = np.zeros(numMRTs)
name_list = [None]*numMRTs
savename_list = [None]*numMRTs
sup_list = [None]*numMRTs

#Render the different scenes with both SRT and MRT
for ai in range(0, numMRTs):
    #Render the scene with the SRT
    print("Running the SRT...")
    timestart_st = timer.time() #Start the timer
    renderer_st = mrt.Renderer_Standard(scene=scene_list[ai], sampler=None, interper=None) #Generate the SRT
    resst_list[ai] = renderer_st.render_scene(form_shade="matte")["image"] #Run the SRT
    timetotst_list[ai] = timer.time() - timestart_st #Stop the timer
    print("Finished running the SRT!")

    #Render the scene with the MRT
    print("Running #{0} of 0-{1} MRTs...".format(ai, (numMRTs-1)))
    timestart_in = timer.time() #Start the timer
    #Generate sampler and interpolator
    sampler = mrt.Sampler_Fraction(valdict={"form":"grid", "fraction":frac}) #Fixed grid of samples
    interper = mrt.Interper_RBF(valdict={"verbose":True, "form":"linear", "max_inpoints":2000, "num_cappoints":10}) #Linear RBF
    #Generate and run the MRT
    renderer_in = mrt.Renderer_Interpolation(scene=scene_list[ai], sampler=sampler, interper=interper,
                                                verbose=False, dofilter=True,
                                                sigma=sigma, cutoff=cutoff) #Generate the MRT
    resin_list[ai] = renderer_in.render_scene(form_shade="matte")["image"] #Run the MRT
    timetotin_list[ai] = timer.time() - timestart_in #Stop the timer
    print("Done running the {0}th MRT!".format(ai))
    
    #Store info strings for graphing later
    name_list[ai] = "Results for {0}th #Lights Test ({1} Lights).".format(ai, count_list[ai])
    savename_list[ai] = "{0}".format(ai)
    sup_list[ai] = ("{0}x{1} grid. Pix. size={2}. Sample Frac.={3:.3f}. Filter Sigma={4:.2f}, Filter Cut={5:.2f}."
                    .format(hspan, vspan, pixsize, frac, sigma, cutoff))

In [ ]:
##Display performance results.
for ai in range(0, numMRTs):
    fig = plt.figure(figsize=(10,4))
    plt.axis("off")
    plt.suptitle("Results for #Lights Test ({0} Lights)".format(count_list[ai]))
    plt.title(sup_list[ai])
    
    print("Time for standard ray-tracer: {0}s.".format(timetotst_list[ai]))
    #Graph the SRT results
    ax0 = fig.add_subplot(1, 3, 1)
    pltmap = plt.imshow(resst_list[ai], extent=imxy, origin='lower')
    plt.title("SRT: {0:.2f}s.".format(timetotst_list[ai]))
    ax0.xaxis.set_visible(False)
    ax0.yaxis.set_visible(False)
    
    #Graph the MRT results
    print("Time for {0}th modified ray-tracer: {1}s.".format(ai, timetotin_list[ai]))
    #Graph the final image
    ax0 = fig.add_subplot(1, 3, 2)
    pltmap = plt.imshow(resin_list[ai], extent=imxy, origin='lower')
    plt.title("MRT: {0:.2f}s.".format(timetotin_list[ai]))
    ax0.xaxis.set_visible(False)
    ax0.yaxis.set_visible(False)
    
    #Graph the error
    ax0 = fig.add_subplot(1, 3, 3)
    errgrid = np.sum(np.abs(resin_list[ai] - resst_list[ai]), axis=2)
    sumhere = errgrid.sum()
    score_list[ai] = sumhere
    pltmap = plt.imshow(errgrid/3.0, cmap=plt.cm.gnuplot2_r, origin='lower', norm=logger(vmin=1E-3, vmax=1))
    cbar = plt.colorbar(pltmap, cax=axloc(ax0).append_axes("right", size="5%", pad=0))
    cbar.set_label(r"$\sum_c$|Error|/3", rotation=270, labelpad=20)
    ax0.xaxis.set_visible(False)
    ax0.yaxis.set_visible(False)
    if dosave:
        plt.savefig("{0}_{1}.png".format(saveroot, savename_list[ai]))
        plt.close()
    else:
        plt.show()
    
#Graph the overall performance
#Plot the error
plt.title(r"Error/3N$_\mathrm{pix}$ vs. Number of Lights")
plt.plot(count_list, (score_list/1.0/(3*hspan*vspan)), marker="o", color="black", linewidth=3, alpha=0.6)
plt.xlabel("Number of Lights")
plt.ylabel(r"$\sum_{x,y,c}$|Error|/3N$_\mathrm{pix}$")
plt.tight_layout()
if dosave:
    plt.savefig(saveroot+"_performance.png")
    plt.close()
else:
    plt.show()
#Plot the time
plt.title("Time vs. Number of Lights")
plt.semilogy(count_list, (timetotin_list/1.0/timetotst_list), marker="o", color="blue", linewidth=3, alpha=0.6)
plt.ylim([0.01, 1])
plt.xlabel("Number of Lights")
plt.ylabel(r"(MRT Time / SRT Time)")
plt.tight_layout()
if dosave:
    plt.savefig(saveroot+"_time.png")
    plt.close()
else:
    plt.show()